In [1]:
from __future__ import print_function

import sys
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

from uartlite import *

import multiprocessing

# For sharing string variable
from multiprocessing import Process,Manager,Value
from ctypes import c_char_p

import asyncio

ROM_SIZE = 0x2000 #8K

In [2]:
ol = Overlay("caravel_fpga.bit")
#ol.ip_dict

In [3]:
ipOUTPIN = ol.output_pin_0
ipPS = ol.caravel_ps_0
ipReadROMCODE = ol.read_romcode_0
ipUart = ol.axi_uartlite_0

In [4]:
ol.interrupt_pins

{'axi_uartlite_0/interrupt': {'controller': 'axi_intc_0',
  'index': 0,
  'fullpath': 'axi_uartlite_0/interrupt'},
 'axi_intc_0/intr': {'controller': 'axi_intc_0',
  'index': 0,
  'fullpath': 'axi_intc_0/intr'}}

In [5]:
# See what interrupts are in the system
#ol.interrupt_pins

# Each IP instances has a _interrupts dictionary which lists the names of the interrupts
#ipUart._interrupts

# The interrupts object can then be accessed by its name
# The Interrupt class provides a single function wait 
# which is an asyncio coroutine that returns when the interrupt is signalled.
intUart = ipUart.interrupt

In [6]:
# Create np with 8K/4 (4 bytes per index) size and be initiled to 0
rom_size_final = 0

npROM = np.zeros(ROM_SIZE >> 2, dtype=np.uint32)
npROM_index = 0
npROM_offset = 0
fiROM = open("integrate.hex", "r+")
#fiROM = open("counter_wb.hex", "r+")

for line in fiROM:
    # offset header
    if line.startswith('@'):
        # Ignore first char @
        npROM_offset = int(line[1:].strip(b'\x00'.decode()), base = 16)
        npROM_offset = npROM_offset >> 2 # 4byte per offset
        #print (npROM_offset)
        npROM_index = 0
        continue
    #print (line)

    # We suppose the data must be 32bit alignment
    buffer = 0
    bytecount = 0
    for line_byte in line.strip(b'\x00'.decode()).split():
        buffer += int(line_byte, base = 16) << (8 * bytecount)
        bytecount += 1
        # Collect 4 bytes, write to npROM
        if(bytecount == 4):
            npROM[npROM_offset + npROM_index] = buffer
            # Clear buffer and bytecount
            buffer = 0
            bytecount = 0
            npROM_index += 1
            #print (npROM_index)
            continue
    # Fill rest data if not alignment 4 bytes
    if (bytecount != 0):
        npROM[npROM_offset + npROM_index] = buffer
        npROM_index += 1
    
fiROM.close()

rom_size_final = npROM_offset + npROM_index
#print (rom_size_final)

#for data in npROM:
#    print (hex(data))


In [7]:
# Allocate dram buffer will assign physical address to ip ipReadROMCODE

#rom_buffer = allocate(shape=(ROM_SIZE >> 2,), dtype=np.uint32)
rom_buffer = allocate(shape=(rom_size_final,), dtype=np.uint32)

# Initial it by npROM
#for index in range (ROM_SIZE >> 2):
for index in range (rom_size_final):
    rom_buffer[index] = npROM[index]
    
#for index in range (ROM_SIZE >> 2):
#    print ("0x{0:08x}".format(rom_buffer[index]))

# Program physical address for the romcode base address


# 0x00 : Control signals
#        bit 0  - ap_start (Read/Write/COH)
#        bit 1  - ap_done (Read/COR)
#        bit 2  - ap_idle (Read)
#        bit 3  - ap_ready (Read)
#        bit 7  - auto_restart (Read/Write)
#        others - reserved
# 0x10 : Data signal of romcode
#        bit 31~0 - romcode[31:0] (Read/Write)
# 0x14 : Data signal of romcode
#        bit 31~0 - romcode[63:32] (Read/Write)
# 0x1c : Data signal of length_r
#        bit 31~0 - length_r[31:0] (Read/Write)

ipReadROMCODE.write(0x10, rom_buffer.device_address)
ipReadROMCODE.write(0x1C, rom_size_final)

ipReadROMCODE.write(0x14, 0)

# ipReadROMCODE start to move the data from rom_buffer to bram
ipReadROMCODE.write(0x00, 1) # IP Start
while (ipReadROMCODE.read(0x00) & 0x04) == 0x00: # wait for done
    continue
    
print("Write to bram done")


Write to bram done


In [8]:
# Initialize AXI UART
uart = UartAXI(ipUart.mmio.base_addr)

# Setup AXI UART register
uart.setupCtrlReg()

# Get current UART status
uart.currentStatus()

{'RX_VALID': 0,
 'RX_FULL': 0,
 'TX_EMPTY': 1,
 'TX_FULL': 0,
 'IS_INTR': 0,
 'OVERRUN_ERR': 0,
 'FRAME_ERR': 0,
 'PARITY_ERR': 0}

In [18]:
async def uart_rx():
    # Reset FIFOs, enable interrupts
    ipUart.write(CTRL_REG, 1<<RST_TX | 1<<RST_RX | 1<<INTR_EN)
    print("Waitting for interrupt")
    while(True):
        await intUart.wait()
        # Read FIFO until valid bit is clear
        while ((ipUart.read(STAT_REG) & (1<<RX_VALID))):
            uart_val = ipUart.read(RX_FIFO)
            if(uart_val == ord('\n')):
                print()
            else:
                print(f"{uart_val:02x}", end=' ')

async def uart_tx():
    await asyncio.sleep(1)
    ipUart.write(CTRL_REG, 1<<RST_TX | 1<<RST_RX | 1<<INTR_EN)
    tx_str = "Done Homework\n"
    i = 0

    while (True):
        if(i<len(tx_str)):
            ipUart.write(TX_FIFO, ord(tx_str[i]))
            await intUart.wait()
            while ((ipUart.read(STAT_REG) & (1<<TX_FULL))):
                pass
            i=i+1
            await asyncio.sleep(0.5)
        else:
            break

                
        
async def caravel_start():
    ipOUTPIN.write(0x10, 0)
    print("Start Caravel Soc")
    ipOUTPIN.write(0x10, 1)

# Python 3.5+
#tasks = [ # Create a task list
#    asyncio.ensure_future(example1()),
#    asyncio.ensure_future(example2()),
#]
# To test this we need to use the asyncio library to schedule our new coroutine. 
# asyncio uses event loops to execute coroutines. 
# When python starts it will create a default event loop 
# which is what the PYNQ interrupt subsystem uses to handle interrupts

#loop = asyncio.get_event_loop()
#loop.run_until_complete(asyncio.wait(tasks))

# Python 3.7+
async def async_main(): 
    task2 = asyncio.create_task(caravel_start()) 
    task3 = asyncio.create_task(uart_tx())
    task1 = asyncio.create_task(uart_rx())
    
    # Wait for 5 second
    await asyncio.sleep(15)
    task1.cancel()
    task3.cancel()
    try:
        await task1
        await task3
    except asyncio.CancelledError:
        print('main(): uart_rx is cancelled now')

In [19]:
asyncio.run(async_main()) 

Start Caravel Soc
Waitting for interrupt
ab 11 
00 3e 
00 44 
00 4a 
00 50 
ab 19 
ab 21 
00 28 
03 7d 
09 ed 

6d 
0c a1 
10 ab 
12 0e 
16 31 
17 87 
23 71 
ab 29 
ab 31 
00 00 
ff f6 
ff e3 
ff e7 
00 23 
00 9e 
01 51 
02 1b 
02 dc 
03 93 
04 4a 
ab 39 
44 6f 6e 65 20 48 6f 6d 65 77 6f 72 6b 
main(): uart_rx is cancelled now


# Address Info
---
## caravel_ps_control_s_axi
File: ../vitis_prj/hls_caravel_ps/hls_caravel_ps.prj/solution1/impl/verilog/caravel_ps_control_s_axi.v
``` c
// 0x00 : reserved
// 0x04 : reserved
// 0x08 : reserved
// 0x0c : reserved
// 0x10 : Data signal of ps_mprj_in
//        bit 31~0 - ps_mprj_in[31:0] (Read/Write)
// 0x14 : Data signal of ps_mprj_in
//        bit 5~0 - ps_mprj_in[37:32] (Read/Write)
//        others  - reserved
// 0x18 : reserved
// 0x1c : Data signal of ps_mprj_out
//        bit 31~0 - ps_mprj_out[31:0] (Read)
// 0x20 : Data signal of ps_mprj_out
//        bit 5~0 - ps_mprj_out[37:32] (Read)
//        others  - reserved
// 0x24 : Control signal of ps_mprj_out
//        bit 0  - ps_mprj_out_ap_vld (Read/COR)
//        others - reserved
// 0x34 : Data signal of ps_mprj_en
//        bit 31~0 - ps_mprj_en[31:0] (Read)
// 0x38 : Data signal of ps_mprj_en
//        bit 5~0 - ps_mprj_en[37:32] (Read)
//        others  - reserved
// 0x3c : Control signal of ps_mprj_en
//        bit 0  - ps_mprj_en_ap_vld (Read/COR)
//        others - reserved
// (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)
```

## output_pin_control_s_axi
File: ../vitis_prj/hls_output_pin/hls_output_pin.prj/solution1/impl/verilog/output_pin_s_axi.v
``` c
// 0x00 : reserved
// 0x04 : reserved
// 0x08 : reserved
// 0x0c : reserved
// 0x10 : Data signal of outpin_ctrl
//        bit 0  - outpin_ctrl[0] (Read/Write)
//        others - reserved
// 0x14 : reserved
// (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)
```

## read_romcode_control_s_axi
File: ../vitis_prj/hls_read_romcode/hls_read_romcode.prj/solution1/impl/verilog/read_romcode_control_s_axi.v
``` c
// 0x00 : Control signals
//        bit 0  - ap_start (Read/Write/COH)
//        bit 1  - ap_done (Read/COR)
//        bit 2  - ap_idle (Read)
//        bit 3  - ap_ready (Read/COR)
//        bit 7  - auto_restart (Read/Write)
//        bit 9  - interrupt (Read)
//        others - reserved
// 0x04 : Global Interrupt Enable Register
//        bit 0  - Global Interrupt Enable (Read/Write)
//        others - reserved
// 0x08 : IP Interrupt Enable Register (Read/Write)
//        bit 0 - enable ap_done interrupt (Read/Write)
//        bit 1 - enable ap_ready interrupt (Read/Write)
//        others - reserved
// 0x0c : IP Interrupt Status Register (Read/COR)
//        bit 0 - ap_done (Read/COR)
//        bit 1 - ap_ready (Read/COR)
//        others - reserved
// 0x10 : Data signal of romcode
//        bit 31~0 - romcode[31:0] (Read/Write)
// 0x14 : Data signal of romcode
//        bit 31~0 - romcode[63:32] (Read/Write)
// 0x18 : reserved
// 0x1c : Data signal of length_r
//        bit 31~0 - length_r[31:0] (Read/Write)
// 0x20 : reserved
// (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)
```

In [11]:
print ("0x10 = ", hex(ipPS.read(0x10))) # Data signal of ps_mprj_in[31:0]
print ("0x14 = ", hex(ipPS.read(0x14))) # Data signal of ps_mprj_in[37:32]
print ("0x1c = ", hex(ipPS.read(0x1c))) # Data signal of ps_mprj_out[31:0]
print ("0x20 = ", hex(ipPS.read(0x20))) # Data signal of ps_mprj_out[37:32]
print ("0x34 = ", hex(ipPS.read(0x34))) # Data signal of ps_mprj_en[31:0]
print ("0x38 = ", hex(ipPS.read(0x38))) # Data signal of ps_mprj_en[37:32]

0x10 =  0x0
0x14 =  0x0
0x1c =  0xab390040
0x20 =  0x0
0x34 =  0x20
0x38 =  0x3f
